# How educator compensation has changed over time

This analyis is based on this NCES report (http://nces.ed.gov/pubsearch/pubsinfo.asp?pubid=2016156) on instructional staff compensation from 1991 to 2011. This report presents nationwide figures, so I wanted to break out Connecticut's data for a  story on TrendCT.org using the following raw data:

* Nonfiscal data: https://nces.ed.gov/ccd/stnfis.asp
* Fiscal data: https://nces.ed.gov/ccd/stfis.asp
* Data documentation here: http://nces.ed.gov/ccd/pdf/stfis111agen.pdf

#### A note for nontechnical readers
I encourage nontechnical readers to look over my notebooks. There is a lot of code (in gray boxes), but sticking to just the headlines and copy text should still be decipherable for anyone trying to better understand my methodology, including the error checking I did and questions I set out to answer.

The reason there is so much code before I get to any conclusions is that the data is stored in a weirdly named zip file containing a spreadsheet for each year on the raw data web pages linked above. In order to download and expand all of those files without losing track of which file went with which year, I had to write a lot of code.

#### A note for technical readers
I tried and gave up on a different approach to importing this data, which you can see in the other notebooks in this directory. They might be useful, but I'm not making their documentation a priority.

# Importing data

It took a lot of work to import all these sheets, so this is a neater presentation than the in the notebooks I used to figure out how to do it. Skip down to the Analysis portion if you want to just see the calculations for the story

In [1]:
%matplotlib inline

import pandas as pd, numpy as np, re

In [2]:
fiscal_links = pd.read_csv("data/2/fiscal_links.csv")
nonfiscal_links = pd.read_csv("data/2/nonfiscal_links.csv")

In [3]:
import urllib2, zipfile, os

In [4]:
def download(zip_url, path):
    response = urllib2.urlopen(zip_url)
    zipcontent= response.read()
    
    with open("tmp.zip", 'w') as f:
        f.write(zipcontent)
        f.close()
    

    zip_ref = zipfile.ZipFile("tmp.zip", 'r')
    zip_ref.extractall(path)
    zip_ref.close()

In [5]:
def import_excel_file(dir_path):
    #print "Looking for excel files in " + dir_path
    excel_files = []
    for filename in os.listdir(dir_path):
        if ".xls" in filename.lower():
            excel_files.append(dir_path + "/" + filename)
            
    if len(excel_files) == 0: 
        return None
    elif len(excel_files) == 1:
        ret_frame = pd.read_excel(excel_files[0])#[["SURVYEAR","STABR","E11","E12"]]
        #ret_frame["STABR"] = ret_frame["STABR"].apply(lambda x: str(x).upper().strip())
        ret_frame.columns = map(lambda x: str(x).strip().upper(), ret_frame.columns)
        return ret_frame
    else:
        throw("Error: Multiple excel files found in " + dir_path)

In [6]:
import re
def clean_fiscal_colname(col_name):
    return  re.sub("FISCAL_YEAR_","",re.sub(r"[^\w]+","_",re.sub(r"[\(\)]","",col_name).strip().upper()))

In [7]:
def get_frameset(folder_path, links_frame):
    frame_set = {}
    for row, val in links_frame.iterrows():
        try:
            year = val["School year"]
            zip_link = val["Excel Link_link"]
        except:
            print "Error"
            print val
        path =  folder_path + str(year)
        #print path
        #download(zip_link, path)
        frame_set[clean_fiscal_colname(year)] = import_excel_file(path)
        
    return frame_set

In [8]:
fiscal_frames = get_frameset("data/2/fiscal/", fiscal_links)
nonfiscal_frames = get_frameset("data/2/nonfiscal/", nonfiscal_links)


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [9]:
# Need this to translate state names to postal abbreviations for 
# the couple of years when postals weren't used
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [10]:
# This structure describes which columns contain the desired data in each file
# since a couple of them don't match
good_years = {
    "nonfiscal":{
        "2013":{
            "sheet_name":'2013_14_V_1A',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
         "2012":{
            "sheet_name":'2013_14_V_1A',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2011":{
            "sheet_name": '2011_12_V_1A',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2010":{
            "sheet_name": '2010_11_V_1A',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2009":{
            "sheet_name": '2009_10_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2008":{
            "sheet_name": '2008_09_V_1C',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2007":{
            "sheet_name":  '2007_08_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2006":{
            "sheet_name":   '2006_07_V_1C',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2005":{
            "sheet_name": '2005_06_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2004":{
            "sheet_name": '2004_05_V_1F',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2003":{
            "sheet_name": '2003_04_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2002":{
            "sheet_name": '2002_03_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2001":{
            "sheet_name":  '2001_02_V_1C',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2000":{
            "sheet_name":  '2000_01_V_1C',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1999":{
            "sheet_name": '1999_2000_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1998":{
            "sheet_name": '1998_99_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1997":{
            "sheet_name": '1997_98_V_1C',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1996":{
            "sheet_name": '1996_97_V_1C',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1995":{
            "sheet_name": '1995_96_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1994":{
            "sheet_name": '1994_95_V_1B',
            "state_column":"STATE",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1993":{
            "sheet_name": '1993_94_V_1B',
            "state_column":"STATE",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1992":{
            "sheet_name": '1992_93_V_1C',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1991":{
            "sheet_name": '1991_92_V_1C',
            "state_column":"STABR",
            "teacher_column":'TOTAL TEACHERS', 
            "aides_column":'INSTRUCTIONAL AIDES',
        }
    },
    "fiscal":{
        "2013":{
            "sheet_name": '2013_V_1A_PROVISIONAL',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",
        },
        "2012":{
            "sheet_name": '2012_V_1A_PROVISIONAL',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",
        },
        "2011":{
            "sheet_name":'2011_V_1A_PRELIMINARY',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2010":{
            "sheet_name":'2010_V_1A_PROVISIONAL',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2009":{
            "sheet_name":'2009_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2008":{
            "sheet_name":'2008_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2007":{
            "sheet_name":'2007_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2006":{
            "sheet_name":'2006_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2005":{
            "sheet_name":'2005_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2004":{
            "sheet_name":'2004_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2003":{
            "sheet_name":'2003_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2002":{
            "sheet_name":'2002_V_1D_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2000":{
            "sheet_name":'2000_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        } ,
        "2001":{
            "sheet_name":'2001_V_1B_REVISED',#'2000_V_1A_PROVISIONAL',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1999":{
            "sheet_name":'1999_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1998":{
            "sheet_name":'1998_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1997":{
            "sheet_name":'1997_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1996":{
            "sheet_name":'1996_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1995":{
            "sheet_name":'1995_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1994":{
            "sheet_name":'1994_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1993":{
            "sheet_name":'1993_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1992":{
            "sheet_name":'1992_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1991":{
            "sheet_name":'1991_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        } 
    }
}

def get_year_nonfiscal(year):
    record = good_years["nonfiscal"][year]
    sheet_name = record["sheet_name"]
    state_column = record["state_column"]
    teacher_column = record["teacher_column"]
    aides_column = record["aides_column"]

    try:
        fr = nonfiscal_frames[sheet_name][[state_column,teacher_column,aides_column]]
        fr[state_column] = fr[state_column].apply(lambda x: str(x).upper().strip())
        fr.columns = "STATE","TEACHERS","AIDES"
    
    except:
        print nonfiscal_frames[sheet_name].columns
    
    #fr["YEAR"] = year
    return fr

def get_year_fiscal(year):
    record = good_years["fiscal"][year]
    sheet_name = record["sheet_name"]
    
    state_column = record["state_column"]
    E11_column = record["E11_column"]
    E12_column = record["E12_column"]

    fr = fiscal_frames[sheet_name][[state_column,E11_column,E12_column]]
    fr[state_column] = fr[state_column].apply(lambda x: str(x).upper().strip())
    fr.columns = "STATE","E11","E12"
    #fr["YEAR"] = year
    return fr

def postal(state_name):
    if state_name.title() in us_state_abbrev:
        return us_state_abbrev[state_name.title()]
    return state_name

def get_year(year):
    left = get_year_fiscal(year).set_index("STATE")
    right = get_year_nonfiscal(year)
    
    # Convert state names to postal abbreviations
    right["STATE"] = right["STATE"].apply(lambda x: postal(x))
    right = right.set_index("STATE")

    
    fr = left.join(right)
    fr["E11"] = pd.to_numeric(fr["E11"], errors="coerce")
    fr = fr[fr["E11"].notnull()]
    fr["year"] = year
    return fr

In [11]:
def get_state_year(state, year):
    fr = get_year(str(year))
    #fr = fr[fr["STATE"] == state]
    return fr.loc[state]

def get_state (state):
    record = {}
    total_dollars = 0
    total_educators = 0
    for year in range(1991,2014):
        record[year] = get_state_year(state, year)[["E11","E12","TEACHERS","AIDES"]]
        continue
        row = {
            "DOLLARS": fr["E11"],
            "EDUCATORS": fr["TEACHERS"]
        }
        
        row = {
            state: float(fr["E11"]) / float(fr["TEACHERS"])
        }
        
        try:
            #row = (float(fr["E11"]) + float(fr["E12"])) / (float(fr["TEACHERS"]) + float(fr["AIDES"]))
            row = (float(fr["E11"]) + float(fr["E12"]))
        except:
            row = None
            
        record[year] = row
        
    return record

In [12]:
all_states = {}
for state_name in us_state_abbrev.keys():
    stabr = us_state_abbrev[state_name]
    print "Getting:", stabr

    all_states[stabr] = get_state(stabr)

Getting: MS


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:313: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:328: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

Getting: OK
Getting: DE
Getting: MN
Getting: IL
Getting: AR
Getting: NM
Getting: IN
Getting: LA
Getting: TX
Getting: WI
Getting: KS
Getting: CT
Getting: CA
Getting: WV
Getting: GA
Getting: ND
Getting: PA
Getting: AK
Getting: MO
Getting: SD
Getting: CO
Getting: NJ
Getting: WA
Getting: NY
Getting: NV
Getting: MD
Getting: ID
Getting: WY
Getting: AZ
Getting: IA
Getting: MI
Getting: UT
Getting: VA
Getting: OR
Getting: MT
Getting: NH
Getting: MA
Getting: SC
Getting: VT
Getting: FL
Getting: HI
Getting: KY
Getting: RI
Getting: NE
Getting: OH
Getting: AL
Getting: NC
Getting: TN
Getting: ME


In [13]:
# Return all the postal abbreviations of US states as a list
def postals():
    ret = []
    for state_name in us_state_abbrev:
        ret.append(us_state_abbrev[state_name])
    return ret
all_postals = postals()

# Analysis

Here's the numer-crunching. It's still code-heavy, so I'll put clear subheds on the output I used in the story for charts and copy.

In [14]:
def sum_col(year, states, col):
    total = 0
    for state in states:
        try: 
            total += float(all_states[state][year][col])
        except:
            print "ERROR Converting to numeric value: " + all_states[state][year][col] + " in ", state , str(year), " " + col
    return total

def sum_salaries(year, states):
    return sum_col(year, states, "E11")

def sum_benefits(year, states):
    return sum_col(year, states, "E12")

def compensation(year, states):
    return sum_salaries(year, states) + sum_benefits(year, states)


def sum_aides (year, states):
    return sum_col (year,states,"AIDES")
def sum_teachers (year, states):
    return sum_col(year, states,"TEACHERS")
def educators(year, states):
    return sum_teachers(year, states) + sum_aides(year, states)

def per_teacher(year, states):
    return compensation (year, states) / educators(year, states)

In [15]:
def all_years(f, state):
    ret = {}
    values = []
    indexes = []
    for year in range(1991, 2014):
        indexes.append(year)
        values.append(int(f(year, state)))
        ret[year] = f(year, state)
    return indexes, values

In [16]:
from uscpi import UsCpi

def inflate(dollars, from_year, to_year=2011):
    cpi = UsCpi()
    return int(cpi.value_with_inflation(dollars, from_year, to_year))

def inflater_2011(row):
    #print "inflating", row
    #print "---------"
    year = row["year"]
    old_dollars = row["amount"]
    #print "inflating", old_dollars, " in ", str(year)
    cpi = UsCpi()
    return int(cpi.value_with_inflation(old_dollars, year, 2011))

def pay_timeline(cols):
    pay_timeline = all_years(per_teacher, cols)
    fr = pd.DataFrame(pay_timeline[1],index=pay_timeline[0]).reset_index()
    fr.columns = "year","amount"
    fr["2011 dollars"] = fr.apply(inflater_2011, axis=1)
    return fr
    #return fr.to_csv(index=False,sep="\t")
    #return fr[["year","2011 dollars"]].to_csv(index=False,sep="\t")
#inflater_2011(1991, 100)

def csvframe(fr):
    return fr.to_csv(index=False,sep="\t")

# * Pay over time, adjusted for inflation

The following two tables show the compensation (salary and benefits) of instructional staff. The first one is CT, and the second is nationwide. It shows both the nominal dollar value in the "amount" column" and the 2011-adjusted dollars.

Note that for this and subsequent 2011-adjusted tables, the data in years after 2011 doesn't necessarily make sense

In [17]:
print csvframe(pay_timeline(["CT"]).set_index("year")\
.join(pay_timeline(postals()).set_index("year")
                                            ,rsuffix="_US",lsuffix="_CT")\
               .reset_index()[["year","2011 dollars_CT","2011 dollars_US"]])

ERROR Converting to numeric value: M in  NV 1991  AIDES
ERROR Converting to numeric value: M in  NV 1991  AIDES
year	2011 dollars_CT	2011 dollars_US
1991	85229	65691
1992	87325	65912
1993	86267	65923
1994	85898	65683
1995	83550	66264
1996	81927	65217
1997	77978	64954
1998	76769	64978
1999	76468	64834
2000	75994	66090
2001	75854	67082
2002	78152	69165
2003	79941	70331
2004	85464	69920
2005	87813	70287
2006	89609	70805
2007	89216	72315
2008	77384	72870
2009	86274	75385
2010	86559	77612
2011	86971	75547
2012	86386	73136
2013	86882	73171



In [18]:
pay_timeline(postals())

ERROR Converting to numeric value: M in  NV 1991  AIDES
ERROR Converting to numeric value: M in  NV 1991  AIDES


,year,amount,2011 dollars
0,1991,39776,65691
1,1992,41111,65912
2,1993,42349,65923
3,1994,43275,65683
4,1995,44895,66264
5,1996,45491,65217
6,1997,46347,64954
7,1998,47086,64978
8,1999,48019,64834
9,2000,50595,66090


# * Was there really a decline in 2007 to 2008 in CT?

The table in the previos section shows a nominal and real (inflation-adjusted) decline in compensation costs per educator from 2007 to 2008 which caught my eye. 


#### Exec summary
It turns out there was a 17% increase in the number of instructional staff members (up 9,163 from 53045 to 62208), all but 4 of which were teachers, not aides. Meanwhile, the costs grew 5.6% (245 million), split even with 123 million in benefits and 123 million salaries. But that means the smaller benefit line grew at a faster pace than salaries - up 10.7%, compared with salary costs which moved at 3.8%.

#### Checking my work

I wanted to make sure that wasn't an error. In the next chunk of code, I verify that my math and code is right in calculating these columns, and then derive the conclusions in the "Exec summary" subsection above.

#### Check 1: Check the math and code

I see a big increase in the number of teachers

In [19]:
def test_year(year):
    print "========================================"
    print "Testing: ", year
    print "========================================"
    print 
    print "-------------------------"
    print "+ Educatour count tests + "
    print "-------------------------"
    print "Total teachers  : ", sum_teachers(year, ["CT"])
    print "Total aides     : ", sum_aides(year, ["CT"])
    print "Teachers + aides: ", educators(year, ["CT"])
    print "Teachers + aides: ", sum_teachers(year, ["CT"]) + sum_aides(year, ["CT"])
    print
    print "---------------------------"
    print "+ Compensation cost tests +"
    print "---------------------------"
    print "Salary costs  :", sum_salaries(year, ["CT"])
    print "Benefits costs:", sum_benefits(year, ["CT"])
    print "Sal + benefits:", compensation(year,["CT"])
    print "Sal + benefits:", sum_salaries(year, ["CT"]) + sum_benefits(year, ["CT"])
    print
    print "--------------------------"
    print "+ Per-teacher cost tests +"
    print "--------------------------"
    print "Cost per teacher:", per_teacher(year, ["CT"])
    print "Cost per teacher:", (sum_salaries(year, ["CT"]) \
                                + sum_benefits(year, ["CT"])) \
    / (sum_teachers(year, ["CT"]) + sum_aides(year, ["CT"]))

In [20]:
test_year(2007)

Testing:  2007

-------------------------
+ Educatour count tests + 
-------------------------
Total teachers  :  39304.0
Total aides     :  13741.0
Teachers + aides:  53045.0
Teachers + aides:  53045.0

---------------------------
+ Compensation cost tests +
---------------------------
Salary costs  : 3221094679.0
Benefits costs: 1141204140.0
Sal + benefits: 4362298819.0
Sal + benefits: 4362298819.0

--------------------------
+ Per-teacher cost tests +
--------------------------
Cost per teacher: 82237.7004242
Cost per teacher: 82237.7004242


In [21]:
test_year(2008)

Testing:  2008

-------------------------
+ Educatour count tests + 
-------------------------
Total teachers  :  48463.0
Total aides     :  13745.0
Teachers + aides:  62208.0
Teachers + aides:  62208.0

---------------------------
+ Compensation cost tests +
---------------------------
Salary costs  : 3343802807.0
Benefits costs: 1263905104.0
Sal + benefits: 4607707911.0
Sal + benefits: 4607707911.0

--------------------------
+ Per-teacher cost tests +
--------------------------
Cost per teacher: 74069.3787133
Cost per teacher: 74069.3787133


#### Check 2: Check the imported data against the original spreadsheet files

I see that the code and math is working as expected. Now I'm going to manually check the original Excel files and make sure the numbers in the columns line up with the data that was imported. I'll write my findings out to serve as a log as I check the data.

* Fiscal 2007: 
    * Filename: stfis071b.xls
    * E11: 3221094679
    * E12: 1141204140
    * CORRECT
* Fiscal 2008:
    * Filename: stfis081b.xls
    * E11: 3343802807
    * E12: 1263905104
    * CORRECT
* Nonfiscal 2007:
    * Filename: st071b.xls
    * TOTTCH: 39304
    * AIDES: 13741
    * CORRECT
* Nonfiscal 2008
    * Filename: st081c.xls
    * TOTTCH: 48463
    * AIDES: 13745
    * CORRECT


#### It's not wrong. Summarize the data

I confirmed in the above steps that the correct data was imported and computed correctly. So, what makes for the compensation decline? The following code generates a summary of the data

In [22]:
# Check the # and % change from year to year for:
# teacher count, aide count, combined educator count, salary costs and benefit costs
import json

def summarize (v1, v2):
    return {
        "first": v1,
        "second": v2,
        "dif": (v2 - v1),
        "pct": (v2 - v1) * 100 / v1,
    }

def change(year1, year2, states, to_inflate=True):
    summary = {
        "TEACHER COUNT":summarize(sum_teachers(year1,states),sum_teachers(year2,states)),
        "AIDE COUNT":summarize(sum_aides(year1,states),sum_aides(year2,states)),
        "EDUCATOR COUNT":summarize(educators(year1,states),educators(year2,states)),
        "SALARY COSTS":summarize(sum_salaries(year1,states),sum_salaries(year2,states)),
        "BENFT COSTS":summarize(sum_benefits(year1,states),sum_benefits(year2,states)),
        "TOTAL COSTS":summarize(compensation(year1,states),compensation(year2,states))
        }
    
    ## Adjust firsts for inflation
    #if (to_inflate):
    #    summary["SALARY COSTS"]["first"] = inflate( summary["SALARY COSTS"]["first"], year1, year2)
    #   summary["BENFT COSTS"]["first"] = inflate( summary["SALARY COSTS"]["first"], year1, year2)
    #   summary["TOTAL COSTS"]["first"] = inflate( summary["SALARY COSTS"]["first"], year1, year2)

    
    print "Plain-english summary of change from ", year1,"to",year2, " in ", str(states)
    print "--------------------------------------------------"
    for k in summary.keys():
        print "* ", k + " went from ", summary[k]["first"], "to", summary[k]["second"],\
        ". That's a change of", summary[k]["dif"],"or",summary[k]["pct"],"percent."
# return "The number of teachers grew " \
#+ str(dif_teacher) + " ("+ str(pct_teacher) + "%) from " \
#+ str(teachers1) + " to " + str(teachers2)

    return summary

summary = change(1991, 2011, ["CT"])
#print json.dumps(summary, indent=2)

Plain-english summary of change from  1991 to 2011  in  ['CT']
--------------------------------------------------
*  BENFT COSTS went from  460789724.0 to 1448012365.0 . That's a change of 987222641.0 or 214.245802278 percent.
*  EDUCATOR COUNT went from  39930.0 to 58058.01 . That's a change of 18128.01 or 45.3994740796 percent.
*  TEACHER COUNT went from  34383.0 to 43804.81 . That's a change of 9421.81 or 27.4025245034 percent.
*  TOTAL COSTS went from  2060646358.0 to 5049417843.0 . That's a change of 2988771485.0 or 145.040485642 percent.
*  SALARY COSTS went from  1599856634.0 to 3601405478.0 . That's a change of 2001548844.0 or 125.108012897 percent.
*  AIDE COUNT went from  5547.0 to 14253.2 . That's a change of 8706.2 or 156.953308094 percent.


So we can see above that the teacher count grew at a far faster pace (23 %) than the costs (6%). The Aide count barely moved, adding just 4 aides. 

Does that mean more new teachers at lower salaries? Maybe more part-time teachers made up that total FTE count. Can't say from the data.

# * Total spending on instructional staff over time

Here I make sure that the numbers in my data set support the bullet point in (http://nces.ed.gov/pubs2016/2016156.pdf) that says the only year spending on salary and benefit spending (aggregate, not per-teacher cost) declined was from 2010 to 2011. This isn't central to my story, but it would help identify any problems with their or my analysis if I couldn't replicate that conclusion. Indeed, I draw the same conclusion.

In [23]:
# helper function to make tuple pairs into a dataframe
def tupleframe(frame_tuple, columns=False):
    return pd.DataFrame(frame_tuple[1],index=frame_tuple[0]).reset_index()

fr = tupleframe(all_years(compensation, postals()))
fr.columns = "year", "amount"
fr["2011 $"] = fr.apply(inflater_2011, axis=1)
fr

,year,amount,2011 $
0,1991,112806928363,186304534941
1,1992,118394428336,189818419924
2,1993,124847148033,194345969767
3,1994,130637836016,198283024261
4,1995,138587727698,204552394230
5,1996,144571202440,207263873203
6,1997,152864617313,214238094416
7,1998,160698507380,221762954303
8,1999,169347010842,228647942807
9,2000,180967011605,236391048916


In [24]:
fr = tupleframe(all_years(compensation, ["CT"]))
fr.columns = "year", "amount"
fr["2011 $"] = fr.apply(inflater_2011, axis=1)
fr

,year,amount,2011 $
0,1991,2060646358,3403228569
1,1992,2221476305,3561629783
2,1993,2255736547,3511440298
3,1994,2384781232,3619637689
4,1995,2467542282,3642037358
5,1996,2517697877,3609486569
6,1997,2589442842,3629075909
7,1998,2710420451,3740363594
8,1999,2857356593,3857928779
9,2000,3025098500,3951583225


# Summarizing the change from 1991 to 2011, without inflation

Since I built that handy summary tool to check my work from 2007 to 2008, I'll apply it below to summarize the net change from 1991 to 2011, the span of time looked at in the NCES analysis

In [25]:
change(1991, 2011,["CT"])

Plain-english summary of change from  1991 to 2011  in  ['CT']
--------------------------------------------------
*  BENFT COSTS went from  460789724.0 to 1448012365.0 . That's a change of 987222641.0 or 214.245802278 percent.
*  EDUCATOR COUNT went from  39930.0 to 58058.01 . That's a change of 18128.01 or 45.3994740796 percent.
*  TEACHER COUNT went from  34383.0 to 43804.81 . That's a change of 9421.81 or 27.4025245034 percent.
*  TOTAL COSTS went from  2060646358.0 to 5049417843.0 . That's a change of 2988771485.0 or 145.040485642 percent.
*  SALARY COSTS went from  1599856634.0 to 3601405478.0 . That's a change of 2001548844.0 or 125.108012897 percent.
*  AIDE COUNT went from  5547.0 to 14253.2 . That's a change of 8706.2 or 156.953308094 percent.


{'AIDE COUNT': {'dif': 8706.2,
  'first': 5547.0,
  'pct': 156.95330809446548,
  'second': 14253.2},
 'BENFT COSTS': {'dif': 987222641.0,
  'first': 460789724.0,
  'pct': 214.24580227835116,
  'second': 1448012365.0},
 'EDUCATOR COUNT': {'dif': 18128.009999999995,
  'first': 39930.0,
  'pct': 45.399474079639354,
  'second': 58058.009999999995},
 'SALARY COSTS': {'dif': 2001548844.0,
  'first': 1599856634.0,
  'pct': 125.10801289711063,
  'second': 3601405478.0},
 'TEACHER COUNT': {'dif': 9421.809999999998,
  'first': 34383.0,
  'pct': 27.402524503388296,
  'second': 43804.81},
 'TOTAL COSTS': {'dif': 2988771485.0,
  'first': 2060646358.0,
  'pct': 145.04048564164157,
  'second': 5049417843.0}}

# How costs have changed, with inflation

This table shows the total salary plus benefit compensation costs, nominal ("amount") and adjusted ("2011 $") over time. Values for years after 2011 aren't useful

In [26]:
# Adjust the "amount" column in a table with "year" and "amount" columns
def adjust_2011(frame):
    fr = pd.DataFrame(frame[1],
                      index=frame[0]).reset_index()

    fr.columns = "year","amount"
    fr["2011 $"] = fr.apply(inflater_2011, axis=1)
    return fr

adjust_2011(all_years(compensation,["CT"]))

,year,amount,2011 $
0,1991,2060646358,3403228569
1,1992,2221476305,3561629783
2,1993,2255736547,3511440298
3,1994,2384781232,3619637689
4,1995,2467542282,3642037358
5,1996,2517697877,3609486569
6,1997,2589442842,3629075909
7,1998,2710420451,3740363594
8,1999,2857356593,3857928779
9,2000,3025098500,3951583225


# How much did the cost of salaries and benefits change from 1991 to 2011 after accounting for inflation?

Here's the two pertinant years from the table above:

| Year | Nominal | 2011 $ |
|---|---|---|
| 1991| 2060646358 |	3403228569
| 2011 | 5049417843 | 5049417843 | 

From the below computation we see that that's a 48% increase.

In [27]:
summarize(3403228569, 5049417843)

{'dif': 1646189274, 'first': 3403228569, 'pct': 48, 'second': 5049417843}

In [28]:
# The same calculation utilizing functions made for previos work
summarize(inflate(compensation(1991,["CT"]),1991),
         compensation(2011, ["CT"]))

{'dif': 1646189274.0,
 'first': 3403228569,
 'pct': 48.37139911774172,
 'second': 5049417843.0}

#### Break out the salaries change over time

From the below computation we see salaries went from 2642218439 to 3601405478 (in 2011 dollars), a 36% increase

In [29]:
summarize(inflate(sum_salaries(1991,["CT"]),1991),
         sum_salaries(2011, ["CT"]))

{'dif': 959187039.0,
 'first': 2642218439,
 'pct': 36.3023368863864,
 'second': 3601405478.0}

#### Break out benefits change over time

In [30]:
summarize(inflate(sum_benefits(1991,["CT"]),1991),
         sum_benefits(2011, ["CT"]))

{'dif': 687002235.0,
 'first': 761010130,
 'pct': 90.27504469618559,
 'second': 1448012365.0}

#### Break out the change in teachers over time

In [31]:
summarize(sum_teachers(1991,["CT"]),
         sum_teachers(2011, ["CT"]))

{'dif': 9421.809999999998,
 'first': 34383.0,
 'pct': 27.402524503388296,
 'second': 43804.81}

#### Break out the per educator cost change over time


In [32]:
summarize(inflate(per_teacher(1991,["CT"]),1991),
         per_teacher(2011, ["CT"]))

{'dif': 1742.941391032873,
 'first': 85229,
 'pct': 2.0450097866135626,
 'second': 86971.94139103287}

#### Break out the change in aides over time


In [33]:
summarize(sum_aides(1991,["CT"]),
         sum_aides(2011, ["CT"]))

{'dif': 8706.2, 'first': 5547.0, 'pct': 156.95330809446548, 'second': 14253.2}

In [34]:
summarize(educators(1991,["CT"]),
         educators(2011, ["CT"]))

{'dif': 18128.009999999995,
 'first': 39930.0,
 'pct': 45.399474079639354,
 'second': 58058.009999999995}

#### Proportion of aides to all educators in 1991 and 2011


In [35]:
sum_aides(1991,["CT"]) * 100 / educators(1991,["CT"])

13.891810668670173

In [36]:
sum_aides(2011,["CT"]) * 100 / educators(2011,["CT"])

24.549928597277106

#### The number of aides has increased far faster than the number of teachers

This is intersting. If aides make up a larger share of the workforce, one would expect that to bring down the cost per instructional staff member, assuming aides are paid less than teachers. This could account for why the cost per employee has basically treaded water with inflation.

We can see from the computation below that the number of aides did grow far faster than teachers nationwide, but not nearly as fast as the rate in Connecticut.

In [37]:
print "Change in teacher count nationwide"
print summarize(sum_teachers(1991,postals()),
         sum_teachers(2011, postals()))
print
print "Change in aide count nationwide"
print summarize(sum_aides(1991,postals()),
         sum_aides(2011, postals()))
print
print "Change in combined educator count nationwide"
print summarize(educators(1991,postals()),
         educators(2011, postals()))

Change in teacher count nationwide
{'second': 3096984.4900000007, 'dif': 671087.4900000007, 'pct': 27.66347829277173, 'first': 2425897.0}

Change in aide count nationwide
ERROR Converting to numeric value: M in  NV 1991  AIDES
{'second': 708931.5899999999, 'dif': 298817.58999999985, 'pct': 72.86207981195469, 'first': 410114.0}

Change in combined educator count nationwide
ERROR Converting to numeric value: M in  NV 1991  AIDES
{'second': 3805916.0800000005, 'dif': 969905.0800000005, 'pct': 34.19962334419721, 'first': 2836011.0}


#### What's the proportion of aides to all educators nationwide

In [38]:
sum_aides(1991,postals()) * 100 / educators(1991,postals())

ERROR Converting to numeric value: M in  NV 1991  AIDES
ERROR Converting to numeric value: M in  NV 1991  AIDES


14.460945320733947

In [39]:
sum_aides(2011,postals()) * 100 / educators(2011,postals())

18.62709463630632

In [40]:
tupleframe(all_years(sum_aides, ["CT"]))

,index,0
0,1991,5547
1,1992,6592
2,1993,6178
3,1994,6822
4,1995,7520
5,1996,7506
6,1997,8881
7,1998,9950
8,1999,10544
9,2000,10954


# Now let's do the same thing for 1993 to 2013

Since more recent data is avaialable

In [41]:
print "Salary cost change"
print summarize(inflate(sum_salaries(1993,["CT"]),1993, 2013),
         sum_salaries(2013, ["CT"]))
print
print "Benefits cost change"
print summarize(inflate(sum_benefits(1993,["CT"]),1993, 2013),
         sum_benefits(2013, ["CT"]))
print
print "Combined cost change"
print summarize(inflate(compensation(1993,["CT"]),1993, 2013),
         compensation(2013, ["CT"]))
print
print "Per instructor cost change"
print summarize(inflate(per_teacher(1993,["CT"]),1993, 2013),
         per_teacher(2013, ["CT"]))



Salary cost change
{'second': 3671907651.0, 'dif': 693296125.0, 'pct': 23.275815558634886, 'first': 2978611526}

Benefits cost change
{'second': 1633961459.0, 'dif': 975966628.0, 'pct': 148.32436092495686, 'first': 657994831}

Combined cost change
{'second': 5305869110.0, 'dif': 1669262752.0, 'pct': 45.90166181522141, 'first': 3636606358}

Per instructor cost change
{'second': 89979.19896187288, 'dif': 637.198961872884, 'pct': 0.7132132276789013, 'first': 89342}


# Timeline of salaries and benefits


In [42]:
def inflate_timeline_2011(f, states):
    df = tupleframe(all_years(f, states))
    df.columns = "year", "amount"
    df["2011 $"] = df.apply(inflater_2011, axis=1)
    df = df.drop("amount", axis=1)
    return df
    
def compensation_timeline(states):
    total = inflate_timeline_2011(compensation, states)
    benefits = inflate_timeline_2011(sum_benefits,states)
    salaries = inflate_timeline_2011(sum_salaries,states)

    total.columns="year","total"
    benefits.columns="year","benefits"
    salaries.columns="year","salaries"

    total = total.set_index("year")
    benefits = benefits.set_index("year")
    salaries = salaries.set_index("year")

    compensation_timeline = benefits\
    .join(salaries)\
    .join(total)

    compensation_timeline["ben_pct"] = compensation_timeline["benefits"] \
    * 100 / compensation_timeline["total"]
    compensation_timeline["sal_pct"] = compensation_timeline["salaries"] \
    * 100 / compensation_timeline["total"]
    
    return compensation_timeline

ct_compensation_timeline = compensation_timeline(["CT"])
us_compensation_timeline = compensation_timeline(postals())

#ct_salaries = tupleframe(all_years(compensation, ["CT"]))
#ct_salaries.columns = "year", "amount"
#ct_salaries["2011 $"] = fr.apply(inflater_2011, axis=1)
#ct_salaries = ct_salaries.drop("amount", axis=1)
#ct_salaries

In [43]:
# This will do for charting the percentage of benefits relative to the overall
# compensation costs over time
print csvframe(ct_compensation_timeline[["ben_pct"]]\
.join(us_compensation_timeline[["ben_pct"]],rsuffix="_US",lsuffix="_CT").reset_index())

year	ben_pct_CT	ben_pct_US
1991	22.3614169478	19.7715844854
1992	18.0832318416	19.9505942896
1993	18.0936501287	20.8281138428
1994	18.2522981515	20.9608909295
1995	18.7697498077	21.0152134884
1996	18.285998476	20.904632785
1997	18.5746773257	20.8765489825
1998	19.0712905864	20.2909175261
1999	19.4292924245	20.0752775505
2000	19.9480655251	19.9782964544
2001	20.4778801982	20.3861474278
2002	20.895191838	20.5260465915
2003	21.0872486943	21.3890603136
2004	22.0666464843	22.4573606591
2005	24.9307288235	23.5161694668
2006	26.2947047593	24.0243290532
2007	26.160613639	24.5967081256
2008	27.4302349017	25.1241440899
2009	27.8557091331	25.0316989211
2010	27.8977446171	25.6100635631
2011	28.676818002	26.1299992415
2012	30.8137131238	26.7881310423
2013	30.7953593443	27.2335475174



# Timeline of share of aides

The final table needed to make the last chart: The percentage of aides relative to all teaching staff over time.

In [44]:
def teachers_timeline(states):
    teachers = tupleframe(all_years(sum_teachers, states))
    aides = tupleframe(all_years(sum_aides, states))
    total = tupleframe(all_years(educators, states))
    
    teachers.columns="year","teachers"
    aides.columns="year","aides"
    total.columns="year","total"
    
    teachers = teachers.set_index("year")
    aides = aides.set_index("year")
    total = total.set_index("year")
    
    timeline = teachers.join(aides).join(total)
    timeline ["pct_aides"] = timeline["aides"] * 100 / timeline["total"]
    return timeline.reset_index()
    
ct_teachers_timeline = teachers_timeline(["CT"])
us_teachers_timeline = teachers_timeline(postals())


ERROR Converting to numeric value: M in  NV 1991  AIDES
ERROR Converting to numeric value: M in  NV 1991  AIDES
ERROR Converting to numeric value: M in  NV 1991  AIDES
ERROR Converting to numeric value: M in  NV 1991  AIDES


In [45]:
print csvframe(ct_teachers_timeline[["year","pct_aides"]].set_index("year")\
.join(us_teachers_timeline[["year","pct_aides"]].set_index("year"),
     rsuffix="_US",lsuffix="_CT").reset_index())

year	pct_aides_CT	pct_aides_US
1991	13.8918106687	14.4609453207
1992	16.1628049528	14.8250224753
1993	15.1778694969	15.2697864788
1994	16.1896625374	15.6683519094
1995	17.2516632255	16.001998136
1996	17.0370202238	16.2321884161
1997	19.0829197018	16.8710249284
1998	20.4219859612	17.2210513115
1999	20.8994866306	17.6035562522
2000	21.0661948536	17.9000230377
2001	22.1088942756	18.355186531
2002	22.2099610093	17.9352740919
2003	21.445389992	18.3493724551
2004	24.640270307	18.6241680212
2005	23.9348346909	18.0791035684
2006	25.4796243022	18.3069078429
2007	25.9044207748	18.3217332637
2008	22.0952289095	18.5527564753
2009	24.8759219687	18.7593410511
2010	26.6885145169	19.0955626891
2011	24.5495883427	18.6270795257
2012	26.3265894483	19.1581845467
2013	26.3265894483	19.1581845467



# Part 2: Salaries versus benefits

Let's look at how inflation-adjusted per-educator salaries have changed over time in CT and nationwide, then look at the same timeline for benefits.

In [46]:
# Combine the compensation timeline with the educator count timeline
def combo_timeline(states):
    df = compensation_timeline(states).join(teachers_timeline(states).set_index("year"),
                                               lsuffix="_comp",
                                               rsuffix="_teach").reset_index()
    # Add "per-capita" benefits and salaries columns. 
    # These are based on inflation-adjusted figures
    df["comp_percap"] = df["total_comp"] / df["total_teach"]
    df["sal_percap"] = df["salaries"] / df["total_teach"]
    df["ben_percap"] = df["benefits"] / df["total_teach"]
    return df

In [47]:
#ct_combo = ct_compensation_timeline.join(ct_teachers_timeline.set_index("year"),
#                                            lsuffix="_comp",
#                                           rsuffix="_teach").reset_index()

ct_combo_timeline = combo_timeline (["CT"])
ct_combo_timeline

,year,benefits,salaries,total_comp,ben_pct,sal_pct,teachers,aides,total_teach,pct_aides,comp_percap,sal_percap,ben_percap
0,1991,761010130,2642218439,3403228569,22.361417,77.638583,34383,5547,39930,13.891811,85229.866491,66171.260681,19058.605810
1,1992,644057771,2917572011,3561629783,18.083232,81.916768,34193,6592,40785,16.162805,87326.953120,71535.417703,15791.535393
2,1993,635347722,2876092575,3511440298,18.093650,81.906350,34526,6178,40704,15.177869,86267.696000,70658.720887,15608.975088
3,1994,660667063,2958970625,3619637689,18.252298,81.747702,35316,6822,42138,16.189663,85899.608168,70220.955551,15678.652594
4,1995,683601300,2958436057,3642037358,18.769750,81.230250,36070,7520,43590,17.251663,83552.130259,67869.604428,15682.525809
5,1996,660030659,2949455909,3609486569,18.285998,81.714001,36551,7506,44057,17.037020,81927.652110,66946.362871,14981.289216
6,1997,674089140,2954986768,3629075909,18.574677,81.425323,37658,8881,46539,19.082920,77979.241260,63494.848793,14484.392445
7,1998,713335610,3027027983,3740363594,19.071291,80.928709,38772,9950,48722,20.421986,76769.500308,62128.565802,14640.934485
8,1999,749568264,3108360515,3857928779,19.429292,80.570708,39907,10544,50451,20.899487,76468.826763,61611.474797,14857.351965
9,2000,788264411,3163318814,3951583225,19.948066,80.051934,41044,10954,51998,21.066195,75994.907977,60835.393938,15159.514039


In [48]:
us_combo_timeline = combo_timeline (postals())
us_combo_timeline

ERROR Converting to numeric value: M in  NV 1991  AIDES
ERROR Converting to numeric value: M in  NV 1991  AIDES
ERROR Converting to numeric value: M in  NV 1991  AIDES
ERROR Converting to numeric value: M in  NV 1991  AIDES


,year,benefits,salaries,total_comp,ben_pct,sal_pct,teachers,aides,total_teach,pct_aides,comp_percap,sal_percap,ben_percap
0,1991,36835358526,149469176414,186304534941,19.771584,80.228416,2425897,410114,2836011,14.460945,65692.458506,52704.018572,12988.439934
1,1992,37869902846,151948517078,189818419924,19.950594,80.049406,2452892,426935,2879827,14.825022,65913.132950,52763.071212,13150.061738
2,1993,40478599832,153867369935,194345969767,20.828114,79.171886,2497845,450153,2947998,15.269786,65924.729178,52193.851534,13730.877644
3,1994,41561888447,156721135814,198283024261,20.960891,79.039109,2545765,472989,3018754,15.668352,65683.730526,51915.835412,13767.895114
4,1995,42987122343,161565271886,204552394230,21.015213,78.984787,2592915,493962,3086877,16.001998,66265.158680,52339.394115,13925.764565
5,1996,43327751589,163936121614,207263873203,20.904633,79.095367,2662131,515857,3177988,16.232188,65218.582702,51584.877480,13633.705221
6,1997,44725520720,169512573695,214238094416,20.876549,79.123451,2741769,556442,3298211,16.871025,64955.848615,51395.309061,13560.539553
7,1998,44997738161,176765216142,221762954303,20.290918,79.709082,2825099,587724,3412823,17.221051,64979.330690,51794.428291,13184.902399
8,1999,45901709132,182746233675,228647942807,20.075278,79.924722,2905821,620813,3526634,17.603556,64834.610795,51818.882729,13015.728066
9,2000,47226904544,189164144372,236391048916,19.978296,80.021704,2936506,640238,3576744,17.900023,66091.128947,52887.247276,13203.881671


In [49]:
print csvframe(ct_combo_timeline[["year","sal_percap","ben_percap"]])

year	sal_percap	ben_percap
1991	66171.2606812	19058.6058102
1992	71535.4177026	15791.5353929
1993	70658.7208874	15608.9750884
1994	70220.9555508	15678.6525939
1995	67869.6044276	15682.5258087
1996	66946.3628708	14981.2892162
1997	63494.8487935	14484.392445
1998	62128.5658019	14640.9344854
1999	61611.4747973	14857.3519653
2000	60835.3939382	15159.514039
2001	60321.972534	15533.6166884
2002	61823.0444162	16330.2889355
2003	63084.6007008	16857.6135306
2004	66605.6856322	18859.2438589
2005	65921.2083948	21892.6298994
2006	66047.2599783	23562.6652822
2007	65877.4095391	23339.7586766
2008	56157.6722447	21226.7207915
2009	62242.8427311	24032.6504274
2010	62412.6449724	24148.6486251
2011	62031.1667298	24940.7896414
2012	59768.7792155	26619.4082622
2013	60127.301338	26756.0359184



In [50]:
summarize(66171,62031)

{'dif': -4140, 'first': 66171, 'pct': -7, 'second': 62031}

In [51]:
summarize(19058,24940)

{'dif': 5882, 'first': 19058, 'pct': 30, 'second': 24940}

In [52]:
print csvframe(us_combo_timeline[["year","sal_percap","ben_percap"]])

year	sal_percap	ben_percap
1991	52704.0185719	12988.4399341
1992	52763.0712116	13150.0617384
1993	52193.8515342	13730.8776437
1994	51915.8354122	13767.895114
1995	52339.3941145	13925.764565
1996	51584.8774803	13633.7052214
1997	51395.3090615	13560.5395531
1998	51794.4282906	13184.9023993
1999	51818.8827293	13015.7280659
2000	52887.2472763	13203.8816712
2001	53407.4283338	13675.6565922
2002	54969.5848666	14197.2081631
2003	55288.4259441	15043.293998
2004	54218.2615673	15702.3163602
2005	53759.0893319	16529.0865572
2006	53795.698882	17010.7819449
2007	54528.8001429	17787.4062024
2008	54562.6850243	18308.1815069
2009	56515.1919075	18870.2591354
2010	57736.7369764	19876.9023705
2011	55807.2488121	19740.6708293
2012	53544.7286192	19591.9490555
2013	53244.6639387	19927.3296411



In [53]:
summarize(52704,55807)

{'dif': 3103, 'first': 52704, 'pct': 5, 'second': 55807}

In [54]:
summarize(12988,19740)

{'dif': 6752, 'first': 12988, 'pct': 51, 'second': 19740}

# Part 3: State-by-state comparison?

This portion is exploratory to see what patterns exist in other states and if it's worth a third story.

In [55]:
def every_state(f):
    for state_name in us_state_abbrev:
        f(us_state_abbrev[state_name])

changes = {}

def change_91_11(abbr):
    
    try:
        summary = change(1991,2011,[abbr])#.map(lambda x: x["pct"])
        #print summary["BENFT COSTS"]["pct"]
        #changes[abbr] = change(1991,2011,[abbr])
        changes[abbr] = pd.DataFrame(summary)
        
    except:
        print "Error generating change report for ", abbr

every_state(change_91_11)

Plain-english summary of change from  1991 to 2011  in  ['MS']
--------------------------------------------------
*  BENFT COSTS went from  137259946.0 to 475160222.0 . That's a change of 337900276.0 or 246.175439993 percent.
*  EDUCATOR COUNT went from  37899.0 to 40292.16 . That's a change of 2393.16 or 6.31457294388 percent.
*  TEACHER COUNT went from  28111.0 to 32006.66 . That's a change of 3895.66 or 13.8581338266 percent.
*  TOTAL COSTS went from  895756534.0 to 2044551168.0 . That's a change of 1148794634.0 or 128.248535221 percent.
*  SALARY COSTS went from  758496588.0 to 1569390946.0 . That's a change of 810894358.0 or 106.90810886 percent.
*  AIDE COUNT went from  9788.0 to 8285.5 . That's a change of -1502.5 or -15.3504290969 percent.
Plain-english summary of change from  1991 to 2011  in  ['OK']
--------------------------------------------------
*  BENFT COSTS went from  225545783.0 to 613229010.0 . That's a change of 387683227.0 or 171.886710469 percent.
*  EDUCATOR COUN

In [56]:
def print_cols_generator(col, row):
    def pc (abbr):
        try:
            print abbr, changes[abbr][col][row]
        except:
            print "ERROR"
    return pc
    
def print_cols(abbr):
    pass
    #print abbr, changes[abbr][key1].loc(key)

every_state(print_cols_generator("BENFT COSTS","pct"))



MS 246.175439993
OK 171.886710469
DE 279.803866587
MN 185.017891865
IL 421.813574394
AR 216.865429527
NM 230.351279398
IN 236.358066483
LA 233.649322033
TX 210.81444863
WI 234.834287152
KS 224.919450205
CT 214.245802278
CA 253.530487388
WV 234.701496527
GA 245.340331788
ND 168.98938729
PA 107.317163316
AK 521.004170789
MO 271.81062781
SD 201.988463202
CO 198.062039042
NJ 371.885119475
WA 165.385777076
NY 278.711211354
NV ERROR
MD 261.866856819
ID 211.496964749
WY 263.675981666
AZ 295.859874785
IA 213.320858542
MI 158.179327598
UT 242.678228302
VA 192.277567889
OR 154.995043255
MT 139.479735221
NH 507.928198964
MA 715.53856862
SC 210.341810511
VT 176.790649519
FL 161.724165482
HI 195.212994569
KY 310.013489552
RI 219.209958503
NE 259.909473722
OH 163.20535618
AL 272.348546248
NC 203.260495605
TN 374.880174638
ME 247.185015589


In [57]:
every_state(print_cols_generator("SALARY COSTS","pct"))

MS 106.90810886
OK 101.507855136
DE 163.718778468
MN 125.791062556
IL 133.33225122
AR 169.012968952
NM 144.87333382
IN 78.7165256152
LA 116.287487218
TX 199.443088499
WI 103.654408097
KS 129.207782004
CT 125.108012897
CA 128.692168785
WV 77.3609086058
GA 202.751354245
ND 107.29447434
PA 123.68649621
AK 112.947869908
MO 124.539974721
SD 108.096333615
CO 140.670604843
NJ 141.007902247
WA 145.723308263
NY 131.187552555
NV ERROR
MD 157.525001921
ID 144.953817873
WY 138.262510072
AZ 170.421799112
IA 114.147987981
MI 61.6188647712
UT 166.518343419
VA 150.647799347
OR 103.517539237
MT 96.136375694
NH 151.625188378
MA 157.400412692
SC 125.217419169
VT 117.522584386
FL 143.391151174
HI 122.663539155
KY 112.746491334
RI 116.280656634
NE 139.763891731
OH 112.147820043
AL 111.446512348
NC 148.530008285
TN 155.719080453
ME 81.4523800217


In [58]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')


In [59]:
#changes["CT"]

In [60]:
#changes["CT"].loc[["first","second"]]#[["ben_pcap","sal_pcap"]].plot()

In [72]:
import matplotlib.ticker as mtick
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import FuncFormatter
from matplotlib.text import Text

def money(x, pos=0):
    # TODO Add commas
    return "$" + str(x / 1000) + "K"

# Rows for state-by-state spreadsheet containing columns
# salaries, benefits, teacher count, aides count,  
states_spreadsheet = [];

def k1000(x, pos=0):
    #return "{:0.1f}K".format(x / 1000)
    if x < 1000:
        return x
    return str(x / 1000) + "K"

# 1991 to 2011
def plot_df(abbr, df, title="",col1="", col2="", formatter="", fsuffix="",path="png/",ylim=None,yticks=[]):
    
    hex_red = "#800000"
    hex_green = "#2E8B57"
    hex_gray = "#555555"

    font = {'family' : 'normal',
            'weight' : 'normal',
            'size'   : 14}

    matplotlib.rc('font', **font)

    
    #print df
    p = df.plot(ylim=ylim,
                figsize=(2.4,2.4),
                legend=False,
               linewidth=2.0)
    
    p.set_xticks([1991,2011])
    p.set_yticks(yticks)
    #p.set_title(title) 
    
    p.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        #bottom='off',      # ticks along the bottom edge are off
        top='off',
        labelbottom='on',
        labelsize=14) # labels along the bottom edge are off
    
    #tick = mtick.StrMethodFormatter(money)
    y = p.yaxis
    #y.set_major_formatter(FormatStrFormatter('$,%03d'))
    y.set_major_formatter(FuncFormatter(money))
    p.tick_params(
        axis="y",
        which="both",
        left="off",
        right="off",
        labelleft="off"
    )

    # Add labels
    df = df.reset_index()
    #print df
    #print
    #print 
    start_benefits_val = int(df[df["index"] == 1991][col2])
    start_salaries_val = int(df[df["index"] == 1991][col1])
    end_benefits_val = int(df[df["index"] == 2011][col2])
    end_salaries_val = int(df[df["index"] == 2011][col1])

    def label_pos(x, y, text,align="left",text_color=hex_gray):
        p.annotate('{}'.format(text), 
                   xy=(x,y), 
                   xytext=(0, 0), 
                   ha=align,
                   textcoords='offset points',
                   color=text_color)
    
    ben_text_color=hex_gray
    sal_text_color=hex_gray
    
    if (start_benefits_val < end_benefits_val):
        ben_text_color = hex_green
    elif (start_benefits_val > end_benefits_val):
        ben_text_color = hex_red
    if (start_salaries_val < end_salaries_val):
        sal_text_color = hex_green
    elif(start_salaries_val > end_salaries_val):
        sal_text_color = hex_red
    
    
    label_pos(1991, start_benefits_val, formatter(start_benefits_val)
              , align="right")
    label_pos(1991, start_salaries_val, formatter(start_salaries_val)
              , align="right")
    label_pos(2011, end_benefits_val, formatter(end_benefits_val), 
              text_color=ben_text_color)
    label_pos(2011, end_salaries_val, formatter(end_salaries_val), 
              text_color=sal_text_color)
    p.set_ylabel("")
    p.set_xlabel("")
    plt.tight_layout()
    fig = p.get_figure()
    fig.savefig(path + abbr + fsuffix + '.png',
                transparent=True,
                bbox_inches='tight')
    plt.close()
    return p 
    
def get_timeline_df (abbr):
    return pd.DataFrame(all_states[abbr]).transpose().loc[1991:2011].reset_index()

def plot_sal_ben(abbr, df):
    
    ret = df #get_timeline_df(abbr)
    
    #ret = pd.DataFrame(all_states[abbr]).transpose().loc[1991:2011].reset_index()
    ret["E11_2011"] = ret.apply(lambda x: inflate(x["E11"], x["index"]), axis=1)
    ret["E12_2011"] = ret.apply(lambda x: inflate(x["E12"], x["index"]), axis=1)
    ret["AIDES_NUM"] = pd.to_numeric(ret["AIDES"], errors="coerce").fillna(0)
    
    ret["SAL_PCAP"] = ret.apply(lambda x: x["E11_2011"] / (x["TEACHERS"] + x["AIDES_NUM"]), axis=1)
    ret["BEN_PCAP"] = ret.apply(lambda x: x["E12_2011"] / (x["TEACHERS"] + x["AIDES_NUM"]), axis=1)
    
    #print inflate(ret["E11"],ret["index"])
    #ret["E11_2011"] = inflate(ret["E11"], int(ret["index"]))
    #ret["E11_2011"] = inflate(ret["E11"], 
    #ret = pd.DataFrame(all_states[abbr]).loc[["E11","E12"]]\
    #.transpose()\
    #.loc[1991:2011]
    #return pd.DataFrame(all[abbr]).loc[["E11","E12"]]
    
    p = plot_df(abbr, ret.set_index("index")[["SAL_PCAP",
                            "BEN_PCAP"]],
                title="Compensation",
                col1="SAL_PCAP",
                col2="BEN_PCAP",
                formatter=money,
                ylim = (0,100000),
                path="viz/png/compensation/",
                yticks=[25000,50000,75000],
                fsuffix="_BS") 
        
    #p.yaxis.grid(color="#999999", linestyle="-", linewidth=1)
    
    #return the change in overall compensation as a percentage
    
    
    start_row = df[df["index"] == 1991]
    end_row = df[df["index"] == 2011]
    
#     print start_row
#     print end_row
    
#     print int(start_row["SAL_PCAP"])
#     print int(start_row["BEN_PCAP"])
#     print int(end_row["SAL_PCAP"])
#     print int(end_row["BEN_PCAP"])

    start = int(start_row["SAL_PCAP"]) + int(start_row["BEN_PCAP"])
    end = int(end_row["SAL_PCAP"]) + int(start_row["BEN_PCAP"])

    return summarize(start, end)

def plot_teach_aid(abbr, df):

    df["AIDES_NUM"] = pd.to_numeric(df["AIDES"], errors="coerce").fillna(0)
    df["TEACHERS_NUM"] = pd.to_numeric(df["TEACHERS"], errors="coerce").fillna(0)
    p = plot_df(abbr, df.set_index("index")[["TEACHERS_NUM",
                        "AIDES_NUM"]],
                title="Staff makeup",
                col1="TEACHERS_NUM",
                col2="AIDES_NUM",
                formatter=k1000,
                path="viz/png/staff_count/",
                #ylim=(0,500000),
                #yticks=[],
                fsuffix="_EC") 
    
    #start = int(start_row["AIDES_NUM"] + start_row["TEACHERS"])
    #end = int(end_row["AIDES_NUM"] + start_row["TEACHERS"])

    #return summarize(start, end)

plot_sal_ben("NY",get_timeline_df("NY"))
plot_teach_aid("NY",get_timeline_df("NY"))

#plot_teach_aid("NY", get_timeline_df("NY"))
#plot_teach_aid("CA", get_timeline_df("CA"))
#plot_teach_aid("RI", get_timeline_df("RI"))
#plot_teach_aid("DE", get_timeline_df("DE"))

In [73]:
state_changes = {}
for abbr in all_states.keys():
        df = get_timeline_df(abbr)
        plot_sal_ben(abbr, df)
        plot_teach_aid(abbr, df)
        #state_changes[abbr] = df.loc[[1991,2011]]
        # print "ERROR: GENEREATING CHART FOR " + 

In [63]:
def state_name(abbr):
    for k in us_state_abbrev:
        if us_state_abbrev[k] == abbr:
            return k
    # fallback to abbreviation
    return abbr

state_name("CT")

'Connecticut'

In [84]:
def package_states():
    ret = []
    for k in all_states:
        obj = {
            "state": k,
            "statename":state_name(k),
            1991: all_states[k][1991].to_dict(),
            2011: all_states[k][2011].to_dict()
        }
        
        # Skip nevada
        if k == "NV":
            continue
            
        ret.append(obj)

    return ret

out_json = open("viz/data/data.json","w")
out_json.write(json.dumps(package_states()))
out_json.close()


In [83]:
nevada = filter(lambda x: x["statename"] == "Nevada", package_states())
nevada

[{1991: {'AIDES': 1325,
   'E11': 386102238.0,
   'E12': 101191806,
   'TEACHERS': 11409.0},
  2011: {'AIDES': 4115.0,
   'E11': 1439112846,
   'E12': 536624181,
   'TEACHERS': 21131.5},
  'state': 'NV',
  'statename': 'Nevada'}]